<a href="https://colab.research.google.com/github/Mahjabeenqamar1/Git/blob/master/Git_clone_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/sarojaerabelli/py-fhe

Cloning into 'py-fhe'...
remote: Enumerating objects: 305, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 305 (delta 202), reused 273 (delta 176), pack-reused 0
Receiving objects: 100% (305/305), 75.12 KiB | 2.68 MiB/s, done.
Resolving deltas: 100% (202/202), done.


In [ ]:
%cd py-fhe/

/content/py-fhe


In [ ]:
! pwd

/content/py-fhe


In [ ]:
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/py-fhe
  Preparing metadata (setup.py) ... done
  Running setup.py develop for py-fhe


In [ ]:
!pytest

============================= test session starts ==============================
platform linux -- Python 3.8.10, pytest-3.6.4, py-1.11.0, pluggy-0.7.1
rootdir: /content/py-fhe, inifile:
plugins: typeguard-2.7.1
collected 70 items                                                             

tests/test_bit_operations.py ..                                          [  2%]
tests/test_crt.py ..                                                     [  5%]
tests/test_matrix_operations.py .......                                  [ 15%]
tests/test_ntt.py ......                                                 [ 24%]
tests/test_number_theory.py .....                                        [ 31%]
tests/test_polynomial.py ..............                                  [ 51%]
tests/test_random_sample.py .                                            [ 52%]
tests/bfv/test_batch_encoder.py ...                                      [ 57%]
tests/bfv/test_bfv_encrypt_decrypt.py .....                        

In [ ]:
!python3 tests/test_polynomial.py

test_add (__main__.TestPolynomial) ... ok
test_base_decompose (__main__.TestPolynomial) ... ok
test_evaluate (__main__.TestPolynomial) ... ok
test_mod (__main__.TestPolynomial) ... ok
test_multiply (__main__.TestPolynomial) ... ok
test_multiply_01 (__main__.TestPolynomial) ... ok
test_multiply_crt (__main__.TestPolynomial) ... ok
test_multiply_fft (__main__.TestPolynomial) ... ok
test_multiply_naive (__main__.TestPolynomial) ... ok
test_rotate (__main__.TestPolynomial) ... ok
test_round (__main__.TestPolynomial) ... ok
test_scalar_multiply (__main__.TestPolynomial) ... ok
test_str (__main__.TestPolynomial) ... ok
test_subtract (__main__.TestPolynomial) ... ok

----------------------------------------------------------------------
Ran 14 tests in 0.012s

OK


In [ ]:
!python3 tests/test_polynomial.py TestPolynomial

test_add (__main__.TestPolynomial) ... ok
test_base_decompose (__main__.TestPolynomial) ... ok
test_evaluate (__main__.TestPolynomial) ... ok
test_mod (__main__.TestPolynomial) ... ok
test_multiply (__main__.TestPolynomial) ... ok
test_multiply_01 (__main__.TestPolynomial) ... ok
test_multiply_crt (__main__.TestPolynomial) ... ok
test_multiply_fft (__main__.TestPolynomial) ... ok
test_multiply_naive (__main__.TestPolynomial) ... ok
test_rotate (__main__.TestPolynomial) ... ok
test_round (__main__.TestPolynomial) ... ok
test_scalar_multiply (__main__.TestPolynomial) ... ok
test_str (__main__.TestPolynomial) ... ok
test_subtract (__main__.TestPolynomial) ... ok

----------------------------------------------------------------------
Ran 14 tests in 0.012s

OK


In [ ]:
!python3 tests/test_polynomial.py TestPolynomial.test_add

test_add (__main__.TestPolynomial) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [ ]:
"""A module to encode integers as plaintext polynomials by using
Chinese Remainder Theorem (CRT) batching.
"""

from util.ntt import NTTContext
from util.plaintext import Plaintext
from util.polynomial import Polynomial

class BatchEncoder:
    """An encoder for several integers to polynomials using Chinese
    Remainder Theorem (CRT) batching.
    We encode N integers using CRT batching, where N is the degree of the
    polynomial that determines the quotient ring. Each polynomial p(x) in
    the ring Z[x]/f(x), where f(x) = x^N + 1 maps to the N-length vector
    [p(a_0), p(a_1), ..., p(a_N)] where a_i are the roots of f(x). This
    uniquely identifies a polynomial in the quotient ring, so we use it
    to define our encoding, where each of these N-length vectors encodes to
    its unique corresponding polynomial in the quotient ring. However, we
    instead use the Fermat Theoretic Transform to obtain a slightly
    modified N-length vector in faster time.
    Attributes:
        degree (int): Degree of polynomial that determines quotient ring.
        ntt (NTTContext): NTTContext object to encode/decode.
    """

    def __init__(self, params):
        """Inits BatchEncoder with the given parameters.
        Args:
            params (Parameters): Parameters including polynomial degree,
                plaintext modulus, and ciphertext modulus.
        """
        self.degree = params.poly_degree
        self.plain_modulus = params.plain_modulus
        self.ntt = NTTContext(params.poly_degree, params.plain_modulus)  

    def encode(self, values):
        """Encodes a list of integers into a polynomial.
        Encodes a N-length list of integers (where N is the polynomial degree)
        into a polynomial using CRT batching.
        Args: 
            values (list): Integers to encode.
        Returns:
            A Plaintext object which represents the encoded value.
        """
        assert len(values) == self.degree, 'Length of list does not equal \
            polynomial degree.'
        coeffs = self.ntt.ftt_inv(values)
        return Plaintext(Polynomial(self.degree, coeffs))

    def decode(self, plain):
        """Decodes a plaintext polynomial.
        Decodes a plaintext polynomial back to a list of integers.
        Args: 
            plain (Plaintext): Plaintext to decode.
        Returns:
            A decoded list of integers.
        """
        result = self.ntt.ftt_fwd(plain.poly.coeffs)
        return [val % self.plain_modulus for val in result]

In [ ]:
"""A module to decrypt for the BFV scheme."""

from util.plaintext import Plaintext

class BFVDecryptor:

    """An object that can decrypt data using BFV given a secret key.
    Attributes:
        poly_degree: Degree of polynomial in quotient ring.
        ciph_modulus: Coefficient modulus in ciphertext space.
        plain_modulus: Coefficient modulus in plaintext space.
        secret_key (SecretKey): Secret key used for encryption.
    """

    def __init__(self, params, secret_key):
        """Generates private/public key pair for BFV scheme.
        Args:
            params (Parameters): Parameters including polynomial degree,
                plaintext modulus, and ciphertext modulus.
            secret_key (SecretKey): Secret key used for decryption.
        """
        self.poly_degree = params.poly_degree
        self.ciph_modulus = params.ciph_modulus
        self.plain_modulus = params.plain_modulus
        self.scaling_factor = params.scaling_factor
        self.secret_key = secret_key

    def decrypt(self, ciphertext, c2=None):
        """Decrypts a ciphertext.
        Decrypts the ciphertext and returns the corresponding plaintext.
        Args:
            ciphertext (Ciphertext): Ciphertext to be decrypted.
        Returns:
            The plaintext corresponding to the decrypted ciphertext.
        """
        (c0, c1) = (ciphertext.c0, ciphertext.c1)
        intermed_message = c0.add(c1.multiply(self.secret_key.s, self.ciph_modulus),
                                  self.ciph_modulus)
        if c2:
            secret_key_squared = self.secret_key.s.multiply(self.secret_key.s, self.ciph_modulus)
            intermed_message = intermed_message.add(c2.multiply(secret_key_squared, self.ciph_modulus), self.ciph_modulus)
        
        intermed_message = intermed_message.scalar_multiply(1 / self.scaling_factor)
        intermed_message = intermed_message.round()
        intermed_message = intermed_message.mod(self.plain_modulus)
        return Plaintext(intermed_message)

In [ ]:
"""A module to encrypt for the BFV scheme."""

from util.ciphertext import Ciphertext
from util.polynomial import Polynomial
from util.random_sample import sample_triangle

class BFVEncryptor:

    """An object that can encrypt data using BFV given a public key.
    Attributes:
        poly_degree: Degree of polynomial in quotient ring.
        coeff_modulus: Coefficient modulus in ciphertext space.
        public_key (PublicKey): Public key used for encryption.
        delta (int): Floor of ciphertext modulus divided by plaintext modulus.
    """

    def __init__(self, params, public_key):
        """Generates private/public key pair for BFV scheme.
        Args:
            params (Parameters): Parameters including polynomial degree,
                plaintext modulus, and ciphertext modulus.
            public_key (PublicKey): Public key used for encryption.
        """
        self.poly_degree = params.poly_degree
        self.coeff_modulus = params.ciph_modulus
        self.public_key = public_key
        self.scaling_factor = int(params.scaling_factor)

    def encrypt(self, message):
        """Encrypts a message.
        Encrypts the message and returns the corresponding ciphertext.
        Args:
            message (Plaintext): Plaintext to be encrypted.
        Returns:
            A ciphertext consisting of a pair of polynomials in the ciphertext
            space.
        """
        p0 = self.public_key.p0
        p1 = self.public_key.p1
        scaled_message = message.poly.scalar_multiply(self.scaling_factor, self.coeff_modulus)

        random_vec = Polynomial(self.poly_degree,
                                sample_triangle(self.poly_degree))
        error1 = Polynomial(self.poly_degree,
                            sample_triangle(self.poly_degree))
        error1 = Polynomial(self.poly_degree,
                            [0] * self.poly_degree)
        error2 = Polynomial(self.poly_degree,
                            sample_triangle(self.poly_degree))
        error2 = Polynomial(self.poly_degree,
                            [0] * self.poly_degree)
        c0 = error1.add(p0.multiply(random_vec, self.coeff_modulus),
                        self.coeff_modulus).add(scaled_message, self.coeff_modulus)
        c1 = error2.add(p1.multiply(random_vec, self.coeff_modulus), self.coeff_modulus)

        return Ciphertext(c0, c1)

In [ ]:
#